In [63]:
from llama_cpp import Llama
import math

In [64]:
llm = Llama.from_pretrained(
	repo_id="TheBloke/phi-2-GGUF",
	filename="phi-2.Q4_K_M.gguf",
    verbose=False,
)

llama_context: n_ctx_per_seq (512) < n_ctx_train (2048) -- the full capacity of the model will not be utilized


In [65]:
def run_llama(user_input):
    prompt = f"""
You are an AI assistant that never solves math questions itself.
Instead, when a question requires calculation, respond only with:
USE_CALCULATOR: <valid Python expression>

Use Python's math module functions for operations like square root, sine, cosine, etc.

For example:

Q: What is 3 + 4?
A: USE_CALCULATOR: 3 + 4

Q: What is the square root of 5?
A: USE_CALCULATOR: math.sqrt(5)

Q: Who is the president of the USA?
A: Joe Biden is the president of the USA.

Q: What is 12 * 12 + 10?
A: USE_CALCULATOR: 12 * 12 + 10

Q: What is the sine of 30 degrees?
A: USE_CALCULATOR: math.sin(math.radians(30))

Q: {user_input}
A:"""

    output = llm(
        prompt,
        max_tokens=100,
        echo=False,
        temperature=0,
        stop=["Q:", "A:"],
    )

    return output['choices'][0]['text'].strip()


    

In [ ]:
def eval_expression(expression):
    try:
        print(f"Evaluating expression: {expression}")
        result = eval(expression, {"__builtins__": None}, {"math": math})
        return result
    except Exception as e:
        return f"Error evaluating expression: {e}"

def parse_response(response):
    print(f"Raw response: {response}")
    if response.startswith("USE_CALCULATOR:"):
        expression = response[len("USE_CALCULATOR:"):].strip()
        try:
            # Evaluate the expression safely
            result = eval_expression(expression)
            return f"Result: {result}"
        except Exception as e:
            return f"Error: {e}"
    else:
        return response

In [67]:
result = run_llama("what is 15 times 34534554 divided by 27")

print(parse_response(result))

Evaluating expression: 
Result: Error evaluating expression: unexpected EOF while parsing (<string>, line 0)
